<a href="https://colab.research.google.com/github/UpLiftL1f3/Emotion_Sentiment_ML/blob/main/Emotion_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upgrade pip tooling first
# %pip install -q --upgrade pip setuptools wheel

# remove any conflicting wheels that may be half-installed
# %pip uninstall -y -q numpy scipy scikit-learn array-api-compat

# install a compatible trio for Python 3.12
# %pip install -q --no-cache-dir --force-reinstall \
#  "numpy==2.1.2" "scipy==1.14.1" "scikit-learn==1.5.2"


In [2]:
# import os, time
# print("Restarting runtime to finalize installs…")
# time.sleep(1)
# os._exit(0)


In [3]:
import numpy, scipy, sklearn
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("sklearn:", sklearn.__version__)

from sklearn.model_selection import train_test_split  # should now work


NumPy: 2.1.2
SciPy: 1.14.1
sklearn: 1.5.2


# Step 1 — Load & sanity-check your CSVs (already in the env)

In [4]:
# Step 1: Load both CSVs safely (no extra installs needed)

import os, re, unicodedata
import pandas as pd
import numpy as np

# Change these if your files are elsewhere
SENTIMENT_CSV = "combined_sentiment_data.csv"
EMOTIONS_CSV  = "combined_emotion.csv"

for path in [SENTIMENT_CSV, EMOTIONS_CSV]:
    assert os.path.exists(path), f"Missing file: {path}. Upload or set the correct path."

# Load as strings; keep options permissive to avoid breaking on weird rows
read_opts = dict(
    dtype=str,
    na_values=["", " ", "NA", "NaN", "nan", None],
    keep_default_na=True,
    on_bad_lines="skip",  # skip malformed rows instead of crashing
    encoding_errors="replace"
)

df_sent_raw = pd.read_csv(SENTIMENT_CSV, **read_opts)
df_emot_raw = pd.read_csv(EMOTIONS_CSV, **read_opts)

# Helper: guess likely text/label columns; override if needed
def guess_cols(df):
    candidates_text  = {"text","tweet","content","sentence","comment","body","review"}
    candidates_label = {"label","sentiment","target","polarity","emotion"}
    text_col = next((c for c in df.columns if c.lower() in candidates_text), df.columns[0])
    label_col = next((c for c in df.columns if c.lower() in candidates_label), df.columns[-1])
    return text_col, label_col

sent_text_col, sent_label_col = guess_cols(df_sent_raw)
emot_text_col, emot_label_col = guess_cols(df_emot_raw)

# Quick peeks

print("Sentiment shape:", df_sent_raw.shape)
print("Sentiment columns:", df_sent_raw.columns.tolist())
print(f"Guessed sentiment text/label: {sent_text_col} / {sent_label_col}")
display(df_sent_raw[[sent_text_col, sent_label_col]].head())
print("\nSentiment label sample counts:")
display(df_sent_raw[sent_label_col].astype(str).str.strip().str.lower().value_counts().head(20))

print("\nEmotion shape:  ", df_emot_raw.shape)
print("Emotion columns:  ", df_emot_raw.columns.tolist())
print(f"Guessed emotion   text/label: {emot_text_col} / {emot_label_col}")
display(df_emot_raw[[emot_text_col, emot_label_col]].head())
print("\nEmotion label sample counts:")
display(df_emot_raw[emot_label_col].astype(str).str.strip().str.lower().value_counts().head(20))


Sentiment shape: (3309, 2)
Sentiment columns: ['sentence', 'sentiment']
Guessed sentiment text/label: sentence / sentiment


,sentence,sentiment
0,So there is no way for me to plug it in here i...,negative
1,"Good case, Excellent value.",positive
2,Great for the jawbone.,positive
3,Tied to charger for conversations lasting more...,negative
4,The mic is great.,positive



Sentiment label sample counts:


,count
sentiment,
positive,1679
negative,1630



Emotion shape:   (422746, 2)
Emotion columns:   ['sentence', 'emotion']
Guessed emotion   text/label: sentence / emotion


,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear



Emotion label sample counts:


,count
emotion,
joy,143067
sad,121187
anger,59317
fear,49649
love,34554
suprise,14972


# Step 2 — Clean missing/corrupt data & normalize labels

In [5]:
# Step 2: Define cleaning and build tidy frames

import re

URL_RE       = re.compile(r"https?://\S+|www\.\S+")
MENTION_RE   = re.compile(r"@\w+")
HASHTAG_RE   = re.compile(r"#(\w+)")
MULTISPACE_RE= re.compile(r"\s+")

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u200b", "")      # zero-width space
    s = s.lower()
    return s

def clean_text_basic(s: str) -> str:
    s = normalize_text(s)
    s = URL_RE.sub(" ", s)           # remove URLs
    s = MENTION_RE.sub(" ", s)       # remove @mentions
    s = HASHTAG_RE.sub(r"\1", s)     # keep hashtag word, drop '#'
    s = MULTISPACE_RE.sub(" ", s).strip()
    return s

def tidy_df(df, text_col, label_col, task_name):
    out = df[[text_col, label_col]].copy()
    out.columns = ["text", "label_raw"]
    # Basic cleanup
    out["text"] = out["text"].astype(str).map(clean_text_basic)
    out["label_raw"] = out["label_raw"].astype(str).str.strip()
    # Drop missing
    before = len(out)
    out = out.dropna(subset=["text","label_raw"])
    # Remove extremely short text (<=1 token)
    out["__len"] = out["text"].str.split().str.len()
    out = out[out["__len"] > 1].drop(columns="__len")
    # Drop exact duplicates (text+label)
    out = out.drop_duplicates(subset=["text","label_raw"])
    out["task"] = task_name
    out = out.reset_index(drop=True)
    print(f"[{task_name}] kept {len(out):,} rows (from {before:,})")
    return out

df_sent_tidy = tidy_df(df_sent_raw, sent_text_col, sent_label_col, "sentiment")
df_emot_tidy = tidy_df(df_emot_raw, emot_text_col, emot_label_col, "emotion")

# Normalize sentiment labels to 3-way when possible
def map_sentiment(lbl: str) -> str:
    t = lbl.strip().lower()
    if t in {"pos","positive","+1","1","favorable","positif","posi"}:
        return "positive"
    if t in {"neg","negative","-1","unfavorable","negatif"}:
        return "negative"
    if t in {"neu","neutral","0","neutre"}:
        return "neutral"
    return t  # leave as-is; we’ll see final unique set next

df_sent_tidy["label"] = df_sent_tidy["label_raw"].map(map_sentiment)

# If after mapping we have exactly {positive, negative, neutral}, great.
# If we only have two classes and one is "neutral", drop neutral to make it binary.
uniq = set(df_sent_tidy["label"].unique())
if len(uniq) == 2 and "neutral" in uniq:
    df_sent_tidy = df_sent_tidy[df_sent_tidy["label"] != "neutral"].reset_index(drop=True)

# Emotion: just lowercase/trim for consistency (many taxonomies exist)
df_emot_tidy["label"] = df_emot_tidy["label_raw"].astype(str).str.strip().str.lower()

print("\nSentiment classes & counts:")
display(df_sent_tidy["label"].value_counts())

print("\nEmotion classes & counts (top 15):")
display(df_emot_tidy["label"].value_counts().head(15))

# Save cleaned versions to reuse later
df_sent_tidy[["text","label"]].to_csv("cleaned_sentiment.csv", index=False)
df_emot_tidy[["text","label"]].to_csv("cleaned_emotions.csv", index=False)
print("\nSaved cleaned_sentiment.csv and cleaned_emotions.csv")


[sentiment] kept 3,273 rows (from 3,309)
[emotion] kept 416,098 rows (from 422,746)

Sentiment classes & counts:


,count
label,
positive,1659
negative,1614



Emotion classes & counts (top 15):


,count
label,
joy,140776
sad,120981
anger,57226
fear,47659
love,34497
suprise,14959



Saved cleaned_sentiment.csv and cleaned_emotions.csv


# Step 3 — Pre-tokenization prep (no modeling yet)

In [6]:
# Step 3: Make stratified train/val splits we’ll reuse later

from sklearn.model_selection import train_test_split

RNG = 42

# Reload from disk to ensure we truly depend on the cleaned outputs
df_sent = pd.read_csv("cleaned_sentiment.csv", dtype=str)
df_emot = pd.read_csv("cleaned_emotions.csv", dtype=str)

# Drop any late NaNs or empties defensively
df_sent = df_sent.dropna(subset=["text","label"])
df_sent = df_sent[df_sent["text"].str.strip().str.len() > 1]

df_emot = df_emot.dropna(subset=["text","label"])
df_emot = df_emot[df_emot["text"].str.strip().str.len() > 1]

# Stratified splits
sent_train, sent_val = train_test_split(
    df_sent, test_size=0.2, random_state=RNG, stratify=df_sent["label"]
)
emot_train, emot_val = train_test_split(
    df_emot, test_size=0.2, random_state=RNG, stratify=df_emot["label"]
)

print("Sentiment split:", len(sent_train), "train /", len(sent_val), "val")
print("Emotion split:  ", len(emot_train), "train /", len(emot_val), "val")

# (Optional) sanity: show class balance after split
def show_balance(name, train_df, val_df):
    print(f"\n{name} — Train balance:")
    display(train_df["label"].value_counts(normalize=True).round(3))
    print(f"{name} — Val balance:")
    display(val_df["label"].value_counts(normalize=True).round(3))

show_balance("Sentiment", sent_train, sent_val)
show_balance("Emotion",   emot_train, emot_val)


Sentiment split: 2618 train / 655 val
Emotion split:   332878 train / 83220 val

Sentiment — Train balance:


,proportion
label,
positive,0.507
negative,0.493


Sentiment — Val balance:


,proportion
label,
positive,0.507
negative,0.493



Emotion — Train balance:


,proportion
label,
joy,0.338
sad,0.291
anger,0.138
fear,0.115
love,0.083
suprise,0.036


Emotion — Val balance:


,proportion
label,
joy,0.338
sad,0.291
anger,0.138
fear,0.115
love,0.083
suprise,0.036


In [7]:
# Step 4: TF-IDF tokenization (1–2 grams), transform train/val, and save

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
import numpy as np

def fit_tfidf_and_transform(train_texts, val_texts,
                            word_max_features=30000, char_max_features=30000, min_df=2):
    tfidf_word = TfidfVectorizer(
        analyzer='word',
        ngram_range=(1, 2),
        max_features=word_max_features,
        min_df=min_df,
        sublinear_tf=True,
        dtype=np.float32,
        lowercase=True,
    )
    tfidf_char = TfidfVectorizer(
        analyzer='char',
        ngram_range=(3, 5),
        max_features=char_max_features,
        sublinear_tf=True,
        dtype=np.float32,
        lowercase=True,
    )
    vec = FeatureUnion([('word', tfidf_word), ('char', tfidf_char)])

    Xtr = vec.fit_transform(train_texts)   # fit on TRAIN only
    Xva = vec.transform(val_texts)         # transform VAL with same vocab

    vocab_size = sum(len(v.get_feature_names_out()) for _, v in vec.transformer_list)
    density_tr = Xtr.nnz / (Xtr.shape[0] * max(1, Xtr.shape[1]))
    density_va = Xva.nnz / (Xva.shape[0] * max(1, Xva.shape[1]))
    print(f"Combined vocab size (word+char): {vocab_size:,}")
    print(f"Train shape: {Xtr.shape}, density: {density_tr:.6f}")
    print(f"Val   shape: {Xva.shape}, density: {density_va:.6f}")
    return vec, Xtr, Xva
